# Data Preparation: Loading and Cleaning

## Import Libraries

In [1]:
# Import Required Modules and Packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import sys

import matplotlib.pyplot as plt 

In [2]:
cd

/root


## Load Data

In [3]:
# Upload the World Development Indicators Data
wdi = pd.read_csv('Project/Raw/wdi_data.csv', na_values = ['..'])

# Upload the Armed Conflict Location and Event Data
acled = pd.read_csv('Project/Raw/acled_data.csv', error_bad_lines=False)

**World Development Indicators Data (WDI)**   
World Development Indicators (WDI) is the primary World Bank collection of development indicators, compiled from officially recognized international sources. It presents the most current and accurate global development data available, and includes national, regional and global estimates. Data is available on 1,431 development indicators across 217 countries from 1960 to 2019.   
   
The dataset consists of 64 columns and 310,527 rows at the country-series level.
* The first and second column contain country name and code values.
* The third and fourth column contain development indicator series name and code values.
* Columns five to sixty-four contain values for each year corresponding to the respective country and series.

**Armed Conflict Location & Event Data Project (ACLED)**   
The Armed Conflict Location & Event Data Project (ACLED) is a publicly available conflict event dataset designed for disaggregated conflict analysis and crisis mapping. The dataset contains information on the dates and locations of all reported political violence events in over 50 developing countries, with a focus on Africa. The data are drawn from news reports, publications by civil society and human rights organisations, and security updates from international organisations. Data is available on 770,105 conflict events across 149 countries from 1917 to 2020.

The dataset consists of 31 columns and 770,105 at the event-level.
* Columns one to four contain identifying data for each event.
* Columnds five and seven contain data on the time when each event occurred.
* Columns eight to sixteen contain data on the type of event and actors involved.
* Columns eighteen to twenty five and thirty one contain data on the location where each event occurred.
* Columns twenty six to twenty eight contain data on the source of information and any additional notes for each event.
* Column twenty nine contains data on the number of fatalities occurring for each event.
* Column thirty contains the time stamp for each event.

In [4]:
# Shape of Data
print("Shape of Data (WDI):                ",wdi.shape)
print("Shape of Data (ACLED):              ", acled.shape)

# Countries Included
print("Countries Included in Data (WDI):   ", wdi['Country Name'].nunique())
print("Countries Included in Data (ACLED): ", acled['country'].nunique())

Shape of Data (WDI):                 (310527, 64)
Shape of Data (ACLED):               (770105, 31)
Countries Included in Data (WDI):    217
Countries Included in Data (ACLED):  149


## World Development Indicators Data Set    
Data will be cleaned and transformed to the country-year level.

### Transform Data

In [5]:
# Data Dictionary 
wdi_index_series = wdi.loc[:,['Series Code', 'Series Name']].drop_duplicates()
dictionary_series = wdi_index_series.set_index('Series Code').drop_duplicates().T.to_dict(orient='dict')

In [6]:
# Transforming to Country-Year Level
wdi_index = wdi.loc[:,['Country Name', 'Series Code']]
wdi_index.rename(columns={'Country Name':'country',
                          'Series Code':'series'},
                 inplace=True)
wdi_values = wdi.iloc[0:,4:]
wdi_values.columns = wdi_values.columns.str[0:4]
wdi_X_country_series_Y_year = pd.concat([wdi_index, wdi_values], axis=1)
wdi_X_country_series_Y_year = wdi_X_country_series_Y_year.set_index(['country', 'series'])
wdi_X_country_series_Y_year = wdi_X_country_series_Y_year.stack()
wdi_stacked = pd.DataFrame(wdi_X_country_series_Y_year)
wdi_stacked = wdi_stacked.reset_index()
wdi_stacked.rename(columns={ wdi_stacked.columns[2]: "year" }, inplace = True)
wdi_stacked.rename(columns={ wdi_stacked.columns[3]: "value" }, inplace = True)
wdi_stacked = wdi_stacked.set_index(['country', 'series', 'year'])
wdi_unstacked = pd.DataFrame(wdi_stacked.unstack(level=1))
wdi_X_country_year_Y_series = wdi_unstacked.reset_index()
wdi_index_final = wdi_X_country_year_Y_series.iloc[:,:2]
wdi_values_final = wdi_X_country_year_Y_series.iloc[:,2:]
wdi_index_final['year'] = wdi_index_final['year'].astype(float)
wdi_values_final = wdi_values_final.astype(float)
wdi_data = pd.concat([wdi_index_final, wdi_values_final], axis=1)
wdi_data.columns = list(map("".join, wdi_data.columns))
wdi_data.columns = wdi_data.columns.str.replace("value", "")
wdi_data

,country,year,AG.AGR.TRAC.NO,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,...,per_sa_allsa.cov_q4_tot,per_sa_allsa.cov_q5_tot,per_si_allsi.adq_pop_tot,per_si_allsi.ben_q1_tot,per_si_allsi.cov_pop_tot,per_si_allsi.cov_q1_tot,per_si_allsi.cov_q2_tot,per_si_allsi.cov_q3_tot,per_si_allsi.cov_q4_tot,per_si_allsi.cov_q5_tot
0,Afghanistan,1960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1961.0,120.0,NaN,NaN,377000.0,57.745918,7650000.0,0.834296,11.717673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1962.0,150.0,NaN,NaN,377600.0,57.837821,7700000.0,0.823403,11.794259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1963.0,200.0,NaN,NaN,378100.0,57.914407,7750000.0,0.812096,11.870845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1964.0,200.0,NaN,NaN,378730.0,58.010906,7800000.0,0.800428,11.947431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13015,Zimbabwe,2015.0,NaN,144.707741,22.9,162000.0,41.876696,4000000.0,0.289548,10.339925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13016,Zimbabwe,2016.0,NaN,144.707741,22.9,162000.0,41.876696,4000000.0,0.285095,10.339925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13017,Zimbabwe,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13018,Zimbabwe,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean Data

In [7]:
# Cleaning Country Name to Match ACLED Data
wdi_data.loc[wdi_data['country'] == 'Bahamas, The', 'country'] = 'Bahamas'
wdi_data.loc[wdi_data['country'] == 'Congo, Dem. Rep.', 'country'] = 'Democratic Republic of Congo'
wdi_data.loc[wdi_data['country'] == 'Congo, Rep.', 'country'] = 'Republic of Congo'
wdi_data.loc[wdi_data['country'] == "Cote d'Ivoire", 'country'] = 'Ivory Coast'
wdi_data.loc[wdi_data['country'] == 'Egypt, Arab Rep.', 'country'] = 'Egypt'
wdi_data.loc[wdi_data['country'] == 'Gambia, The', 'country'] = 'Gambia'
wdi_data.loc[wdi_data['country'] == 'Iran, Islamic Rep.', 'country'] = 'Iran'
wdi_data.loc[wdi_data['country'] == 'Kyrgyz Republic', 'country'] = 'Kyrgyzstan'
wdi_data.loc[wdi_data['country'] == 'Lao PDR', 'country'] = 'Laos'
wdi_data.loc[wdi_data['country'] == 'Russian Federation', 'country'] = 'Russia'
wdi_data.loc[wdi_data['country'] == 'St. Kitts and Nevis', 'country'] = 'Saint Kitts and Nevis'
wdi_data.loc[wdi_data['country'] == 'St. Lucia', 'country'] = 'Saint Lucia'
wdi_data.loc[wdi_data['country'] == 'St. Martin (French part)', 'country'] = 'Saint-Martin'
wdi_data.loc[wdi_data['country'] == 'St. Vincent and the Grenadines', 'country'] = 'Saint Vincent and Grenadines'
wdi_data.loc[wdi_data['country'] == 'Syrian Arab Republic', 'country'] = 'Syria'
wdi_data.loc[wdi_data['country'] == 'Venezuela, RB', 'country'] = 'Venezuela'
wdi_data.loc[wdi_data['country'] == 'Virgin Islands (U.S.)', 'country'] = 'Virgin Islands, U.S.'
wdi_data.loc[wdi_data['country'] == 'West Bank and Gaza', 'country'] = 'Palestine'
wdi_data.loc[wdi_data['country'] == 'Yemen, Rep.', 'country'] = 'Yemen'

In [8]:
# Dropping data outside the range of 1997 and 2020
wdi_data = wdi_data[(wdi_data.year >= 1997) & (wdi_data.year <= 2020)]

In [9]:
# Shape of Data
print("Shape of Data (WDI): ",wdi_data.shape)

# Countries Included
print("Countries Included in Data (WDI):   ", wdi_data['country'].nunique())

# Years Included
print("Years Included in Data (WDI):   ", wdi_data['year'].nunique())

Shape of Data (WDI):  (4991, 1427)
Countries Included in Data (WDI):    217
Years Included in Data (WDI):    23


## Armed Conflict Location and Event Data Set
Data will be cleaned and transformed to the country-year level.

### Transform Data

In [10]:
# Transforming to Country-Year Level
acled_data = acled[["country", "year", "event_type", "fatalities"]].copy()
acled_data.loc[acled_data['event_type'] == 'Battles', 'event_type'] = 'event_1'
acled_data.loc[acled_data['event_type'] == 'Explosions/Remote violence', 'event_type'] = 'event_2'
acled_data.loc[acled_data['event_type'] == 'Protests', 'event_type'] = 'event_3'
acled_data.loc[acled_data['event_type'] == 'Riots', 'event_type'] = 'event_4'
acled_data.loc[acled_data['event_type'] == 'Strategic developments', 'event_type'] = 'event_5'
acled_data.loc[acled_data['event_type'] == 'Violence against civilians', 'event_type'] = 'event_6'
acled_data = pd.pivot_table(acled_data,index=["country", "year"], columns =['event_type'], aggfunc = [len,np.sum])
acled_data.columns = list(map("_".join, acled_data.columns))
acled_data.columns = acled_data.columns.str.replace("len_fatalities", "conflict_instances")
acled_data.columns = acled_data.columns.str.replace("sum_fatalities", "conflict_fatalities")
acled_data = acled_data.reset_index()
acled_data["year"] = acled_data.year.astype(float)
conflict_instances = ['conflict_instances_event_1', 'conflict_instances_event_2', 'conflict_instances_event_3', 'conflict_instances_event_4', 'conflict_instances_event_5', 'conflict_instances_event_6']
conflict_fatalities = ['conflict_fatalities_event_1', 'conflict_fatalities_event_2', 'conflict_fatalities_event_3', 'conflict_fatalities_event_4', 'conflict_fatalities_event_5', 'conflict_fatalities_event_6']
acled_data['conflict_instances_total'] = acled_data[conflict_instances].sum(axis=1)
acled_data['conflict_fatalities_total'] = acled_data[conflict_fatalities].sum(axis=1)
acled_data

,country,year,conflict_instances_event_1,conflict_instances_event_2,conflict_instances_event_3,conflict_instances_event_4,conflict_instances_event_5,conflict_instances_event_6,conflict_fatalities_event_1,conflict_fatalities_event_2,conflict_fatalities_event_3,conflict_fatalities_event_4,conflict_fatalities_event_5,conflict_fatalities_event_6,conflict_instances_total,conflict_fatalities_total
0,Afghanistan,2017.0,8651.0,3985.0,130.0,5.0,322.0,298.0,26312.0,9591.0,2.0,10.0,247.0,434.0,13391.0,36596.0
1,Afghanistan,2018.0,9546.0,3776.0,215.0,10.0,259.0,332.0,31737.0,10895.0,9.0,5.0,272.0,369.0,14138.0,43287.0
2,Afghanistan,2019.0,9200.0,4008.0,59.0,3.0,242.0,405.0,26516.0,14585.0,5.0,4.0,189.0,388.0,13917.0,41687.0
3,Afghanistan,2020.0,1814.0,749.0,30.0,6.0,93.0,138.0,4063.0,1588.0,0.0,8.0,57.0,114.0,2830.0,5830.0
4,Albania,2018.0,1.0,3.0,209.0,25.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,241.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,eSwatini,2016.0,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,6.0,0.0
1499,eSwatini,2017.0,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,3.0,0.0
1500,eSwatini,2018.0,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,7.0,0.0
1501,eSwatini,2019.0,NaN,NaN,15.0,11.0,NaN,2.0,NaN,NaN,0.0,2.0,NaN,0.0,28.0,2.0


## Clean Data

In [11]:
# Cleaning Country Name to Match WDI Data
acled_data.loc[acled_data.country == 'eSwatini', 'country'] = 'Eswatini'
acled_data.loc[acled_data.country == 'Sint Maarten', 'country'] = 'Saint-Martin'

# Dropping Countries not included in WDI Data
acled_data = acled_data.drop(acled_data[acled_data.country == 'Anguilla'].index)
acled_data = acled_data.drop(acled_data[acled_data.country == 'Caribbean Netherlands'].index)
acled_data = acled_data.drop(acled_data[acled_data.country == 'Falkland Islands'].index)
acled_data = acled_data.drop(acled_data[acled_data.country == 'French Guiana'].index)
acled_data = acled_data.drop(acled_data[acled_data.country == 'Guadeloupe'].index)
acled_data = acled_data.drop(acled_data[acled_data.country == 'Martinique'].index)
acled_data = acled_data.drop(acled_data[acled_data.country == 'Montserrat'].index)

In [12]:
# Shape of Data
print("Shape of Data (ACLED): ",acled_data.shape)

# Countries Included
print("Countries Included in Data (ACLED):   ", acled_data['country'].nunique())

# Years Included
print("Years Included in Data (ACLED):   ", acled_data['year'].nunique())

Shape of Data (ACLED):  (1494, 16)
Countries Included in Data (ACLED):    141
Years Included in Data (ACLED):    24


## Final Data
WDI and ACLED datasets will be merged into a single dataset at the country-year level.

### Merge Data

In [13]:
# WDI & ACLED Countries
wdi_data_countries = wdi_data['country'].unique()
acled_data_countries = acled_data['country'].unique()
print("Countries in ACLED Dataset not listed in WDI Data. These countries will either have their names adjusted to match the WDI Dataset or treated accordingly.:")
list(set(acled_data_countries) - set(wdi_data_countries))
#wdi['Country Name'].nunique()

Countries in ACLED Dataset not listed in WDI Data. These countries will either have their names adjusted to match the WDI Dataset or treated accordingly.:


[]

In [14]:
# Merge WDI and ACLED Data
final_data = pd.merge(wdi_data, acled_data, how='left', left_on=['country', 'year'], right_on=['country', 'year'])
conflict_instances = ['conflict_instances_event_1', 'conflict_instances_event_2', 'conflict_instances_event_3', 'conflict_instances_event_4', 'conflict_instances_event_5', 'conflict_instances_event_6', 'conflict_instances_total']
conflict_fatalities = ['conflict_fatalities_event_1', 'conflict_fatalities_event_2', 'conflict_fatalities_event_3', 'conflict_fatalities_event_4', 'conflict_fatalities_event_5', 'conflict_fatalities_event_6', 'conflict_fatalities_total']
final_data[conflict_instances] = final_data[conflict_instances].replace(np.nan, 0)
final_data[conflict_fatalities] = final_data[conflict_fatalities].replace(np.nan, 0)
final_data = final_data.fillna(method='ffill')
#final_data.set_index(['country', 'year'])
final_data

,country,year,AG.AGR.TRAC.NO,AG.CON.FERT.PT.ZS,AG.CON.FERT.ZS,AG.LND.AGRI.K2,AG.LND.AGRI.ZS,AG.LND.ARBL.HA,AG.LND.ARBL.HA.PC,AG.LND.ARBL.ZS,...,conflict_instances_event_5,conflict_instances_event_6,conflict_fatalities_event_1,conflict_fatalities_event_2,conflict_fatalities_event_3,conflict_fatalities_event_4,conflict_fatalities_event_5,conflict_fatalities_event_6,conflict_instances_total,conflict_fatalities_total
0,Afghanistan,1997.0,110.0,NaN,NaN,377900.0,57.883773,7685000.0,0.397011,11.771283,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Afghanistan,1998.0,110.0,NaN,NaN,378670.0,58.001716,7757000.0,0.393003,11.881567,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Afghanistan,1999.0,110.0,NaN,NaN,377530.0,57.827099,7653000.0,0.379409,11.722268,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Afghanistan,2000.0,110.0,NaN,NaN,377530.0,57.827099,7683000.0,0.369731,11.768220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Afghanistan,2001.0,110.0,NaN,NaN,377530.0,57.827099,7683000.0,0.355579,11.768220,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,Zimbabwe,2015.0,22496.0,144.707741,22.9,162000.0,41.876696,4000000.0,0.289548,10.339925,...,6.0,97.0,0.0,0.0,0.0,12.0,0.0,7.0,264.0,19.0
4988,Zimbabwe,2016.0,22496.0,144.707741,22.9,162000.0,41.876696,4000000.0,0.285095,10.339925,...,14.0,72.0,0.0,0.0,0.0,0.0,0.0,6.0,228.0,6.0
4989,Zimbabwe,2017.0,22496.0,144.707741,22.9,162000.0,41.876696,4000000.0,0.285095,10.339925,...,15.0,151.0,5.0,0.0,0.0,2.0,0.0,2.0,298.0,9.0
4990,Zimbabwe,2018.0,22496.0,144.707741,22.9,162000.0,41.876696,4000000.0,0.285095,10.339925,...,19.0,153.0,5.0,2.0,0.0,10.0,0.0,7.0,344.0,24.0


In [15]:
# Shape of Data
print("Shape of Data (Final): ",final_data.shape)

# Countries Included
print("Countries Included in Data (Final):   ", final_data['country'].nunique())

# Years Included
print("Years Included in Data (Final):   ", final_data['year'].nunique())

Shape of Data (Final):  (4992, 1441)
Countries Included in Data (Final):    217
Years Included in Data (Final):    23


## Save Merged Data

In [16]:
# Save Data
final_data.to_csv('Project/Cleaned/final_data.csv')

# Save Dictionary
np.save('Project/Cleaned/dictionary_series.npy', dictionary_series) 